In [1]:
# Import dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
from citipy import citipy
from datetime import datetime
from config import weather_api_key

In [2]:
# Create a NEW set of random latitude and longitude combinations.
lats = np.random.uniform(-90.000, 90.000, size=2000)
lngs = np.random.uniform(-180.000, 180.000, size=2000)
lat_lngs = zip(lats, lngs)
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [3]:
# Create our city names list.
cities = []
# Identify the nearest city to the lat/lng coords, and save to cities list.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    # Only append unique city names.
    if city not in cities:
        cities.append(city)

In [4]:
# Create building block for API access.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [5]:
# Create a list to store the city data - will make a DF from it later.
city_data = []

# Loop through all the cities in our list - track how far we are getting through the list.
for i, city in enumerate(cities):
    
    # Print a message every 50 cities found - display completed message.
    if i % 50 == 0 and i > 0:
        print(f"Set number {int(i / 50)} of {(len(cities) // 50) + 1} completed.")
        
    # Create the URL to perform the API call with.
    city_url = url + "&q=" + city.replace(" ", "+")
    
    # Perform the API call for every city, and save the required data.
    # Handle any errors involving not being able to find the city.
    try:
        # Save the weather data with JSON format.
        city_weather = requests.get(city_url).json()

        # Parse the data, store all together.
        city_country = city_weather["sys"]["country"]
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        # Create a column for the description of the weather.
        city_description = city_weather["weather"][0]["description"]
        
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_description})
        
    except:
        print(f"Unable to find weather data for: {city} - SKIPPING")

# Print message to show search completion.
print("Set number 16 of 16 completed.")
print("------ SEARCH COMPLETE ------")

Unable to find weather data for: vaitupu - SKIPPING
Unable to find weather data for: barentsburg - SKIPPING
Unable to find weather data for: karamea - SKIPPING
Unable to find weather data for: taolanaro - SKIPPING
Unable to find weather data for: illoqqortoormiut - SKIPPING
Unable to find weather data for: belushya guba - SKIPPING
Unable to find weather data for: sentyabrskiy - SKIPPING
Set number 1 of 16 completed.
Unable to find weather data for: miranorte - SKIPPING
Unable to find weather data for: amderma - SKIPPING
Unable to find weather data for: marcona - SKIPPING
Unable to find weather data for: mys shmidta - SKIPPING
Set number 2 of 16 completed.
Unable to find weather data for: karaul - SKIPPING
Unable to find weather data for: barawe - SKIPPING
Set number 3 of 16 completed.
Unable to find weather data for: handwara - SKIPPING
Unable to find weather data for: saleaula - SKIPPING
Unable to find weather data for: ruatoria - SKIPPING
Unable to find weather data for: tasbuget - S

In [7]:
# Create a DataFrame from the city data from the API calls.
city_df = pd.DataFrame(city_data)
city_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Ushuaia,AR,-54.8000,-68.3000,39.20,93,90,32.21,snow
1,Parintins,BR,-2.6283,-56.7358,87.03,61,89,4.03,heavy intensity rain
2,Punta Arenas,CL,-53.1500,-70.9167,48.20,57,40,20.71,scattered clouds
3,Tahe,CN,52.3333,124.7333,19.81,91,75,3.62,broken clouds
4,Bengkulu,ID,-3.8004,102.2655,76.44,88,96,13.13,overcast clouds
5,Jumla,NP,29.2747,82.1838,39.76,23,18,4.09,few clouds
6,Grafton,US,42.2070,-71.6856,46.99,22,1,11.50,clear sky
7,Nikolskoye,RU,59.7035,30.7861,37.40,75,75,6.71,broken clouds
8,Obo,CF,5.3998,26.4869,82.62,28,99,2.98,overcast clouds
9,Rikitea,PF,-23.1203,-134.9692,77.18,68,55,15.66,broken clouds


In [9]:
# Export the DataFrame to a CSV file for use in other notebooks.
city_df.to_csv("WeatherPy_Database.csv", index_label="City_ID")